In [ ]:
#@title 📦 Download and install rclone binary automatically
import os
import zipfile
import shutil

# Download latest rclone release for Linux amd64
url = "https://downloads.rclone.org/rclone-current-linux-amd64.zip"
zip_path = "/content/rclone.zip"
extract_dir = "/content/rclone_extract"

# Download and unzip
print("⬇️ Downloading rclone...")
!curl -L -o "{zip_path}" "{url}"

print("📦 Extracting...")
os.makedirs(extract_dir, exist_ok=True)
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

# Move the rclone binary to ~/.local/bin
bin_path = os.path.expanduser("~/.local/bin")
os.makedirs(bin_path, exist_ok=True)

# Find the extracted rclone binary path
extracted_dir = next((d for d in os.listdir(extract_dir) if d.startswith("rclone-")), None)
rclone_binary_src = os.path.join(extract_dir, extracted_dir, "rclone")
rclone_binary_dest = os.path.join(bin_path, "rclone")

# Copy and set executable
shutil.copy2(rclone_binary_src, rclone_binary_dest)
os.chmod(rclone_binary_dest, 0o755)

# Update PATH for current session
%env PATH=/root/.local/bin:$PATH

# Confirm installation
print("✅ Rclone installed successfully at ~/.local/bin/rclone\n")
!rclone version


⬇️ Downloading rclone...
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 22.2M  100 22.2M    0     0  11.2M      0  0:00:01  0:00:01 --:--:-- 11.2M
📦 Extracting...
env: PATH=/root/.local/bin:$PATH
✅ Rclone installed successfully at ~/.local/bin/rclone

rclone v1.69.1
- os/version: ubuntu 22.04 (64 bit)
- os/kernel: 6.1.123+ (x86_64)
- os/type: linux
- os/arch: amd64
- go/version: go1.24.0
- go/linking: static
- go/tags: none


In [ ]:
#@title Upload your rclone.conf file
from google.colab import files
import os

print("📁 Upload your rclone.conf (found locally at ~/.config/rclone/rclone.conf)")
uploaded = files.upload()

config_dir = "/root/.config/rclone"
os.makedirs(config_dir, exist_ok=True)

for name in uploaded:
    if name.endswith(".conf"):
        os.rename(name, os.path.join(config_dir, "rclone.conf"))
        print(f"✔️ Uploaded and moved {name} to {config_dir}/rclone.conf")


📁 Upload your rclone.conf (found locally at ~/.config/rclone/rclone.conf)


Saving rclone.conf to rclone.conf
✔️ Uploaded and moved rclone.conf to /root/.config/rclone/rclone.conf


In [ ]:
#@title 🔗 Mount an rclone remote
import os

# Make sure rclone binary and rclone.conf are already uploaded and configured
mount_path = "/content/remote"

# Ask user for remote name (as it appears in rclone.conf)
remote = input("Enter the name of your rclone remote (as in rclone.conf): ")

# Create the mount directory
os.makedirs(mount_path, exist_ok=True)

# Mount the remote
# '--daemon' runs rclone in the background, '--allow-other' lets other processes access it
!rclone mount "{remote}": "{mount_path}" --daemon --allow-other --vfs-cache-mode writes

print(f"✅ Remote '{remote}' mounted at {mount_path}")


In [ ]:
#@title 🎯 Download a torrent and copy it to rclone remote
import libtorrent as lt
import time
import datetime
import shutil
import os

# Define save path (local Colab directory)
local_download_path = "/content/Downloads"
os.makedirs(local_download_path, exist_ok=True)

# Define rclone mount path (must match what you mounted earlier)
rclone_mount_path = "/content/remote"

# Create a torrent session
ses = lt.session()
ses.listen_on(6881, 6891)
ses.start_dht()

# Ask for magnet link
magnet = input("📥 Paste Magnet Link: ").strip()
params = {
    'save_path': local_download_path,
    'storage_mode': lt.storage_mode_t.storage_mode_sparse,
    'flags': lt.add_torrent_params_flags_t.flag_paused  # initially paused so we can select files
}
handle = lt.add_magnet_uri(ses, magnet, params)

print("⏳ Fetching metadata...")
while not handle.has_metadata():
    time.sleep(1)

# Resume now that metadata is available
handle.resume()

# List all files in the torrent
torrent_info = handle.get_torrent_info()
print("\n📂 Files in torrent:")
file_priorities = []
for idx, f in enumerate(torrent_info.files()):
    print(f"{idx}: {f.path} ({f.size / 1e6:.2f} MB)")

# Ask user to select files
selected = input("\nEnter file indices to download (comma-separated, or 'all' for everything): ").strip()
num_files = torrent_info.num_files()

if selected.lower() == "all":
    file_priorities = [1] * num_files  # download all files
else:
    indices = [int(i.strip()) for i in selected.split(",") if i.strip().isdigit() and int(i.strip()) < num_files]
    file_priorities = [1 if i in indices else 0 for i in range(num_files)]

handle.prioritize_files(file_priorities)

print("\n🚀 Starting download...\n")
start = time.time()

# Monitor progress
state_str = ['queued', 'checking', 'downloading metadata', 'downloading', 'finished', 'seeding', 'allocating']
while not handle.status().is_seeding:
    s = handle.status()
    print(f"{s.progress * 100:.2f}% complete "
          f"(↓ {s.download_rate / 1000:.1f} kB/s | ↑ {s.upload_rate / 1000:.1f} kB/s | "
          f"Peers: {s.num_peers}) [{state_str[s.state]}]")
    time.sleep(5)

end = time.time()
print(f"\n✅ Download complete in {int((end - start) // 60)} min {int((end - start) % 60)} sec")

# Copy to rclone remote
dest_path = os.path.join(rclone_mount_path, torrent_info.name())
print(f"\n📁 Copying to remote: {dest_path}")
shutil.copytree(os.path.join(local_download_path, torrent_info.name()), dest_path)

print("\n🎉 All done! Files are now in your rclone remote.")
